In [1]:
import tensorflow as tf
import numpy as np

2023-11-19 00:13:54.737263: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-19 00:13:54.737331: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-19 00:13:54.737351: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-19 00:13:54.777129: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [17]:
NUM_CLASSES = 10
WIDTH = 28
NUM_CHANNELS = 1
NUM_TRAIN = 60000
NUM_TEST = 10000
NUM_DEV = 100

DATASET = "mnist"
BASE_DIR = ""
BATCH_SIZE = 1024

## Data loading

In [7]:
def _normalize(X):
  assert X.dtype == np.uint8
  X = X.astype(np.float64)
  X /= 255
  return X

def load_standard_mnist():
  (X_train, Y_train), (X_validation, Y_validation) = tf.keras.datasets.mnist.load_data(path='mnist.npz')
  X_train = X_train.reshape(X_train.shape[0], WIDTH, WIDTH, 1)
  X_validation = X_validation.reshape(X_validation.shape[0], WIDTH, WIDTH, 1)

  X_train = _normalize(X_train)
  X_validation = _normalize(X_validation)

  Y_train = Y_train.astype(np.int32)
  Y_validation = Y_validation.astype(np.int32)

  return X_train, Y_train, X_validation, Y_validation

def load_mnist_train_dev():
  # these were randomly picked, then fixed for future
  selected_dev = [8106, 9910, 3397, 8870, 2103, 5689, 9799, 4037, 1584, 1160, 9063,
       1332, 3043, 8307, 1042, 3466, 7772, 7327, 7098, 7216, 8624, 6400,
       5811, 1862, 7327, 1626, 5958, 3868, 3795,  836, 3406, 5570, 9535,
       9653, 7890, 5671, 2451, 9175, 8310, 2425, 5923, 2797, 1150, 6012,
       8666, 8849, 6839, 5994, 6751, 9139, 9648, 8898, 9869, 2184, 1363,
       8294, 4000, 5424, 4544,  330, 4325, 4597, 4735, 9966, 2342, 7220,
       5774, 3437, 4276,  760, 7868, 2993, 6262, 8880, 6017, 5045, 9513,
       4084, 7115, 5775,  358, 3549, 2612, 8973, 6747,  415, 8573, 9973,
       2734,  586, 3937, 6889, 1191, 5255, 1460,  413, 7257, 5272, 7402,
       7968]
  X_train, Y_train, X_validation, Y_validation = load_standard_mnist()
  X_dev = X_validation[selected_dev]
  Y_dev = Y_validation[selected_dev]
  return X_train, Y_train, X_dev, Y_dev

In [8]:
X_train, Y_train, X_test, Y_test = load_standard_mnist()
_, _, X_dev, Y_dev = load_mnist_train_dev()

11490434/11490434 [==============================] - 0s 0us/step


In [9]:
Y_train = np.squeeze(np.array(Y_train))
Y_test = np.squeeze(np.array(Y_test))
Y_dev = np.squeeze(np.array(Y_dev))

In [10]:
print(X_train.shape, X_test.shape, X_dev.shape, Y_train.shape, Y_test.shape, Y_dev.shape)

(60000, 28, 28, 1) (10000, 28, 28, 1) (100, 28, 28, 1) (60000,) (10000,) (100,)


## Model training

In [15]:
def deep_model():
  model = tf.keras.Sequential((
      tf.keras.layers.Flatten(input_shape=(WIDTH, WIDTH), name='input', dtype=np.float64),
      tf.keras.layers.Dense(256, name='hidden1', activation='relu', dtype=np.double),
      tf.keras.layers.Dense(128, name='hidden2', activation='relu', dtype=np.double),
      tf.keras.layers.Dense(64, name='hidden3', activation='relu', dtype=np.double),
      tf.keras.layers.Dense(32, name='hidden4', activation='relu', dtype=np.double),
      tf.keras.layers.Dense(NUM_CLASSES, name='logits', dtype=np.float64),
      tf.keras.layers.Activation(tf.nn.softmax, name='softmax', dtype=np.float64)))
  return model

In [1]:
EPOCHS = 10000
saved_epochs = [100, 1000, 5000, 10000]

start_epoch = 0
for e in saved_epochs:
    try:
        clf = tf.keras.models.load_model(f"{BASE_DIR}/{DATASET}_{e}e_{pct_poison}dp.h5")
        start_epoch = e
    except:
        break
    
if start_epoch == 0:
    clf = deep_model()
    clf.compile(
        optimizer='Adam', 
        loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
        metrics=['accuracy', tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5)]
    )

print(start_epoch)
for i in range(start_epoch+1,EPOCHS+1):
  if (i%10) == 0:
    print(i)
  clf.fit(X_train, Y_train, epochs=1, batch_size=BATCH_SIZE, validation_data=(X_test, Y_test), verbose=2)
  if i in saved_epochs:
    clf.save(f"{BASE_DIR}/{DATASET}_{i}e.h5")